In [5]:
from SuperCNN import CNN

from SuperCNN import Parametros

from Auxiliares import takeinputs


In [6]:
val, pix, qcmval, qcmpix, pixelsconv, qcmpixconv = takeinputs()

convlay = [(1, "input"), (5, "relu"), (32, "relu")]

lay = [(64, "sigmoid"), (10, "softmax")]

parametros = Parametros(pix=pix, vales=val, qcmpix=qcmpix, qcmval=qcmval, infolay=lay, infoconvlay=convlay)

In [7]:
g = CNN(parametros)

g.train()


AttributeError: 'CNN' object has no attribute 'convolutionnumba'

In [5]:
print(g.tauxlent())

89.91


In [ ]:
import pickle

with open('parameters.pickle', 'wb') as file:
    pickle.dump(g.parameters, file)
